In [1]:
from tweepy import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
# from tweepy import API
# from tweepy import parsers

from pykafka import KafkaClient
import json

import twitter_credentials

In [18]:
# this is if you want specific user data 

# class TwitterClient():
#     # borrows from TwitterAUthenticator to call API 
    
#     # has two methods twitter_client and auth
    
#     def __init__(self):
        
#         self.auth = TwitterAuthenticator().authenticate_twitter()
#         #JSON parser returns as dictionary
#         self.twitter_client = API(self.auth, parser=parsers.JSONParser())

In [19]:
class TwitterAuthenticator():
    
    # class to authenticate tweets 
    
    # auth method returns credentials 
    
    def auth(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth
        

In [20]:
# create class that allows us to print tweets 
# inherits from Tweepys StreamListener class

# manipulated to print tweets to console

class TwitterListener(StreamListener):
    
    # create constructor 
    # a Listener object thats associated to filename where tweets are writing to
    def __init__(self, tweets_file):
        # a class variable
        self.tweets_file = tweets_file
    
    # here using print statement can use 
    def on_data(self, data):
        try:
            print(data)
            # writes tweets to file, a means appends
            # 
            with open(self.tweets_file, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
    
    def on_error(self, status):
        if status == 420:
            # returns False if rates limit occurs 
            return False
        print(status)


In [21]:
class TwitterStreamer():
    # class for processing streams 
    
    def __init__(self):
        self.twitter_auth = TwitterAuthenticator()
    
    def stream_tweets(self, tweets_file, hash_tags):
        # handles twitter authentication and connection to API
        listener = TwitterListener(tweets_file)
        auth = self.twitter_auth.authenticate_twitter()

        stream = Stream(auth, listener)
        stream.filter(track = hash_tags)

In [22]:
if __name__ == "__main__":
    
    hash_tags = ["covid19", "corona"]
    fetched_tweets_file = "tweets_trial1.json"
    
#     twitter_client = TwitterClient()
#     print(twitter_client)
    
    # define twitter streamer object 
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_file, hash_tags)

In [3]:
# auth = tweepy.OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
# auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)

In [ ]:

#got two options can save text exactly into dictionary and pass as message to consumer 
#or can save whole json to file and read from file 
#https://www.youtube.com/watch?v=xiouigXHqdE

# start kafka, zookeeper and consumer console 
# in terminal 

#zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties
#kafka-server-start.sh $KAFKA_HOME/config/server.properties
#kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic twitterTest --from-beginning


from pykafka import KafkaClient

import twitter
import json

from datetime import datetime
from time import sleep
#import uuid


client = KafkaClient(hosts="localhost:9092")
topic = client.topics['twitterTest']
producer = topic.get_sync_producer()

# with topic.get_sync_producer() as producer:
#     for i in range(4):
#         producer.produce('test message '.encode('ascii') + str(i ** 2).encode('ascii'))

# twitter API

api = twitter.Api(consumer_key='***',
  consumer_secret='***',
    access_token_key='***',
    access_token_secret='***')

data = {}

for i in range(50):
    
    trumptweets = api.GetSearch(term='trump', count=1, result_type='recent', return_json=True)
    
    atweet = trumptweets['statuses'][0]['text']
    
    data['ID'] = i
    data['timestamp'] = str(datetime.utcnow())
    data['tweet'] = atweet
    
    #json.dumps takes object and produces str
    message = json.dumps(data)
    print(message)
    producer.produce(str(message).encode('utf-8'))
    
    sleep(3) #will collect from API every 5 min for the next hour

In [4]:
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [10]:
results = api.search(q="covid19", count=1, tweet_mode="extended", lang="en")

In [11]:
results.keys()

dict_keys(['statuses', 'search_metadata'])

In [52]:
results["search_metadata"];

In [53]:
results["statuses"][0].keys(); #.keys() #["full_text"]

In [31]:
# debugging 


# dict_keys = list(results["statuses"][0].keys())

In [46]:

# for i in dict_keys:
#     print(i)
#     print("")
#     print(results["statuses"][0][i])
#     print("------------------------------------------------------------------------------------------------------")

In [47]:
# results["statuses"][0]["user"]

In [48]:
# change to datetime
results["statuses"][0]["created_at"]

'Tue Apr 14 21:18:49 +0000 2020'

In [113]:
time_stamp = results["statuses"][0]["created_at"]

In [119]:
#ask if want in datetime or str 

str(datetime.datetime.strptime(time_stamp, '%a %b %d %X %z %Y'))

'2020-04-14 21:18:49+00:00'

In [114]:
import datetime
#import strptime

In [49]:
results["statuses"][0]["id"]

1250171676068847617

In [51]:
# clean string
results["statuses"][0]["retweeted_status"]["full_text"]

'1⃣\nWhy Did Iran’s President Hassan #Rouhani End the Quarantine and Ignore the Catastrophe of #Coronavirus Expansion?\n#COVID19 @WHO\n@ForeignOffice @usadarfarsi @StateDept @statedeptspox  @SecPompeo @realDonaldTrump @USUN @mbachelet @javaidRehman @UNHumanRights \n@GermanyDiplo https://t.co/eD8XKisaek'

In [18]:
results["statuses"][0]["user"]["friends_count"]

4244

In [19]:
results["statuses"][0]["user"]["screen_name"]

'exhaustartist'

In [ ]:
# results['statuses'][0]["user"]["friends_count"]
# results['statuses'][0]["user"]["screen_name"] # will be key in Hbase

# full_text, created_at as time_stamp datetime format, id, 

In [54]:
full_tweet = results["statuses"][0]["retweeted_status"]["full_text"]

In [57]:
import re

In [82]:
re.sub("[^'\w+'\w]+", " ", full_tweet)  

'1 Why Did Iran s President Hassan Rouhani End the Quarantine and Ignore the Catastrophe of Coronavirus Expansion COVID19 WHO ForeignOffice usadarfarsi StateDept statedeptspox SecPompeo realDonaldTrump USUN mbachelet javaidRehman UNHumanRights GermanyDiplo https t co eD8XKisaek'

In [90]:
new = re.sub("[^A-Za-z0-9]+'", " ", full_tweet)  

In [112]:
new.replace("\n", "")

'1⃣Why Did Iran’s President Hassan #Rouhani End the Quarantine and Ignore the Catastrophe of #Coronavirus Expansion?#COVID19 @WHO@ForeignOffice @usadarfarsi @StateDept @statedeptspox  @SecPompeo @realDonaldTrump @USUN @mbachelet @javaidRehman @UNHumanRights @GermanyDiplo https://t.co/eD8XKisaek'